In [33]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import matplotlib.lines as mlines

### Hỗn hợp

In [34]:
honhop_folder = "data/Honhop"
data_frames = []
file_names = []
for index, file_name in enumerate(os.listdir(honhop_folder)):
    if file_name.endswith(".CSV"):  # Check if the file is a CSV
        file_path = os.path.join(honhop_folder, file_name)
        df = pd.read_csv(file_path, header=None).values
        if index == 0:
            data_frames.append(df[:, 0].reshape(1, -1))
        data_frames.append(df[:, 1].reshape(1, -1))
        file_names.append(file_name)
honhop = pd.DataFrame(np.concatenate(data_frames[1:]), columns=list(data_frames[0]))
honhop.columns = ['_'.join(map(str, col)) for col in honhop.columns]
honhop["File name"] = file_names
honhop["Type"] = honhop["File name"].str.split("-", expand=True)[0].str[0]
honhop_deriv = savgol_filter(honhop.iloc[:, :-2], window_length=25, polyorder=5, deriv=2)
honhop.iloc[:, :-2] = honhop_deriv

### Rễ

In [35]:
re_folder = "data/Re"
data_frames = []
file_names = []
for index, file_name in enumerate(os.listdir(re_folder)):
    if file_name.endswith(".CSV"):  # Check if the file is a CSV
        file_path = os.path.join(re_folder, file_name)
        df = pd.read_csv(file_path, header=None).values
        if index == 0:
            data_frames.append(df[:, 0].reshape(1, -1))
        data_frames.append(df[:, 1].reshape(1, -1))
        file_names.append(file_name)
re = pd.DataFrame(np.concatenate(data_frames[1:]), columns=list(data_frames[0]))
re.columns = ['_'.join(map(str, col)) for col in re.columns]
re["File name"] = file_names
re["Type"] = re["File name"].str.split("-", expand=True)[0].str[0]
re_deriv = savgol_filter(re.iloc[:, :-2], window_length=25, polyorder=5, deriv=2)
re.iloc[:, :-2] = re_deriv

### Thân

In [36]:
than_folder = "data/Than"
data_frames = []
file_names = []
for index, file_name in enumerate(os.listdir(than_folder)):
    if file_name.endswith(".CSV"):  # Check if the file is a CSV
        file_path = os.path.join(than_folder, file_name)
        df = pd.read_csv(file_path, header=None).values
        if index == 0:
            data_frames.append(df[:, 0].reshape(1, -1))
        data_frames.append(df[:, 1].reshape(1, -1))
        file_names.append(file_name)
than = pd.DataFrame(np.concatenate(data_frames[1:]), columns=list(data_frames[0]))
than.columns = ['_'.join(map(str, col)) for col in than.columns]
than["File name"] = file_names
than["Type"] = than["File name"].str.split("-", expand=True)[0].str[0]
than_deriv = savgol_filter(than.iloc[:, :-2], window_length=25, polyorder=5, deriv=2)
than.iloc[:, :-2] = than_deriv

In [37]:
data_rethan = pd.concat([re, than], axis=0)

In [38]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
scaled_data_rethan = scaler.fit_transform(data_rethan.iloc[:, :-2])
scaled_honhop = scaler.transform(honhop.iloc[:, :-2])

pca = PCA(3)
X_rethan = pca.fit_transform(scaled_data_rethan)
X_honhop = pca.transform(scaled_honhop)
X_re = X_rethan[:re.shape[0], :]
X_than = X_rethan[re.shape[0]:, :]

In [39]:
import plotly.graph_objects as go
import numpy as np

x1 = X_re[:, 0]
y1 = X_re[:, 1]
z1 = X_re[:, 2]

x2 = X_than[:, 0]
y2 = X_than[:, 1]
z2 = X_than[:, 2]

x3 = X_honhop[:, 0]
y3 = X_honhop[:, 1]
z3 = X_honhop[:, 2]

hover_text_re = re["File name"].values
hover_text_than = than["File name"].values
hover_text_honhop = honhop["File name"].values

# Create a 3D scatter plot
fig = go.Figure()
# Add the first scatter trace
fig.add_trace(go.Scatter3d(
    x=x1,
    y=y1,
    z=z1,
    mode='markers',
    marker=dict(
        size=5,
        color='blue',  # Color for the first set
        opacity=0.8
    ),
    hovertemplate = hover_text_re,
    name='Rễ'  # Name for the legend
))

# Add the second scatter trace
fig.add_trace(go.Scatter3d(
    x=x2,
    y=y2,
    z=z2,
    mode='markers',
    marker=dict(
        size=5,
        color='red',  # Color for the second set
        opacity=0.8
    ),
    hovertemplate = hover_text_than,
    name='Thân'  # Name for the legend
))

fig.add_trace(go.Scatter3d(
    x=x3,
    y=y3,
    z=z3,
    mode='markers',
    marker=dict(
        size=5,
        color='orange',  # Color for the second set
        opacity=0.8
    ),
    hovertemplate = hover_text_honhop,
    name='Hỗn hợp'  # Name for the legend
))

# Set the layout of the plot
fig.update_layout(
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis'
    ),
    width = 800,
    height = 600,
    title='Multiple 3D Scatter Plots Example'
)

# Show the plot
fig.show()